In [1]:
import pandas as pd 

In [2]:
import os 
from datetime import datetime

In [3]:
path = "ml-25m"

# Dataset 

In [4]:
rating = pd.read_csv(path + "/" + "ratings.csv")
tag = pd.read_csv(path + "/" + "tags.csv")
movie = pd.read_csv(path + "/" + "movies.csv")


In [6]:
rating

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [7]:
# filmlerin id'leri, isimleri, konuları 
movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [8]:

tag

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455
...,...,...,...,...
1093355,162521,66934,Neil Patrick Harris,1427311611
1093356,162521,103341,cornetto trilogy,1427311259
1093357,162534,189169,comedy,1527518175
1093358,162534,189169,disabled,1527518181


# Features 

**movies**

In [10]:
movie["_genres"] =  movie["genres"].apply(lambda x: x.split("|")[0])

In [11]:
_movie = movie[["movieId", "_genres"]]
_movie

,movieId,_genres
0,1,Adventure
1,2,Adventure
2,3,Comedy
3,4,Comedy
4,5,Comedy
...,...,...
62418,209157,Drama
62419,209159,Documentary
62420,209163,Comedy
62421,209169,(no genres listed)


**tags**

In [ ]:
_tag = tag.groupby(['userId','movieId']).first().reset_index()[["userId",'movieId','tag']]
_tag

**genome_tags**

In [ ]:

genome_scores = pd.read_csv(path + "/" + "genome-scores.csv")
genome_tags = pd.read_csv(path + "/" + "genome-tags.csv")


In [ ]:
genome_tags

In [ ]:
genome_scores

In [ ]:
_genome = pd.merge(genome_scores, genome_tags, on = 'tagId')


In [ ]:
_genome = _genome.groupby('movieId', group_keys=False).apply(lambda x: x.loc[x.relevance.idxmax()]).reset_index(drop=True)

In [ ]:
_genome = pd.merge(_genome, genome_tags, on = ['tagId','tag'], how = 'left').rename(columns = 
                                                                                    {"tag":"genome_tag",
                                                                                    "relevance": "genome_relevance"}
                                                                                   
                                                                                   ).drop(columns = {
    "tagId"
})
_genome

**concat features**

In [ ]:
_genome

In [ ]:
_tag

In [ ]:
_movie

In [ ]:
rating = pd.read_csv(path + "/" + "ratings.csv")
rating

In [ ]:

rating = pd.merge(rating, _movie, on ='movieId', how = 'left')
rating

In [ ]:
rating = pd.merge(rating, _tag, on =['movieId','userId'], how = 'left')
rating

In [ ]:
rating= pd.merge(rating, _genome, on =["movieId"], how = "left")
rating

In [ ]:
rating["tag"] = rating["tag"].fillna("None")
rating["_genres"] = rating["_genres"].fillna("None")
rating["genome_tag"] = rating["genome_tag"].fillna("None")
rating["genome_relevance"] = rating["genome_relevance"].fillna(0)
rating["rating"] = rating["rating"].fillna(0)
rating


In [ ]:
rating._genres.nunique()

In [ ]:
rating.genome_tag.nunique()

In [ ]:
rating.tag.nunique()

In [ ]:
rating["timestamp"] = rating["timestamp"].apply(lambda x: datetime.fromtimestamp(x))

In [ ]:
rating = rating.sort_values(["userId", "timestamp"], ascending =[True, True])

In [ ]:
rating

In [ ]:
rating.to_csv("manipulated_rating.csv", index=False)

In [ ]:
!pwd